In [1]:
import os
from utils.coco_eval import eval, get_mAP
import json
import pandas as pd
identify_files = [file.replace('.pdf','') for file in os.listdir('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/tpbank_benchmark/identify_pdf')]

In [15]:
len(identify_files)


25

In [ ]:
gt_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/groundtruth_coco.json'
pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/model_predictions.json'
filtered_gt_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/filtered_groundtruth_coco.json'
fitlered_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/filtered_model_predictions.json'
# Load the COCO validation annotations
with open(gt_path) as f:
    val_data = json.load(f)

# Load the COCO format predictions
with open(pred_path) as f:
    pred_data = json.load(f)

with open('/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/tpbank_benchmark/scanned_pages.json') as f:
    scanned_pages = json.load(f)
confidence_threshold = 0.5
print(len(pred_data['images']))
print(len(val_data['images']))
pred_data['annotations'] = [annot for annot in pred_data['annotations'] if annot['score'] >=confidence_threshold]
scanned_pages_id = [image['id'] for image in val_data['images'] if image['file_name'] in scanned_pages or image['file_name'].split('_page_')[0] not in identify_files]
val_data['annotations']= [annot for annot in val_data['annotations'] if annot['image_id'] not in scanned_pages_id]
val_data['images'] =[img for img in val_data['images'] if img['id'] not in scanned_pages_id]

pred_data['annotations']= [annot for annot in pred_data['annotations'] if annot['image_id'] not in scanned_pages_id]
pred_data['images'] =[img for img in pred_data['images'] if img['id'] not in scanned_pages_id]
with open(filtered_gt_path, 'w') as f:
    json.dump(val_data,f,ensure_ascii=False,indent=4)
with open(fitlered_pred_path, 'w') as f:
    json.dump(pred_data,f,ensure_ascii=False,indent=4)


In [16]:
gt_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco_v1/test_domain'
pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/layout/rule_based_coco.json'
model_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/model_infer/yolo/'
test_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco/test.json'

In [18]:
metrics_by_domain = {}
for domain in os.listdir(gt_path):
    # Load the COCO validation annotations
    print(domain)
    with open(os.path.join(gt_path,domain)) as f:
        val_data = json.load(f)

    # Load the COCO format predictions
    with open(os.path.join(model_pred_path,domain)) as f:
        model_preds = json.load(f)

    # with open(test_path) as f:
    #     test = json.load(f)
    
    pred_data = model_preds
    pred_data['annotations'] = model_preds['predicted_results'] 
    # {
    #     'images': test['images'],
    #     'annotations': model_preds,
    #     'categories': test['categories']
    # }
    
    domain = 'yolo-' + domain.split('_')[-1].replace('.json','')    
    conf_mat, precision, recall, labels = eval(val_data,pred_data,ignore_image_ids=[],confidence_threshold=0.2)
    mAP = get_mAP(val_data,pred_data,ignore_image_ids=[])
    metrics_by_domain[domain] = {}
    
    for label,prec,rec in zip(labels,precision,recall):
        metrics_by_domain[domain][f'precision-{label}']=prec
        metrics_by_domain[domain][f'recall-{label}']=rec
        
    metrics_by_domain[domain] = {**metrics_by_domain[domain],
                                 **mAP['bbox']}
with open('view/yolo/eval_domain_model.json', 'w') as f:
    json.dump(metrics_by_domain,f, ensure_ascii=False, indent=4)


test_fpt.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
15
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.790
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.875
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.842
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.450
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.508
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.765
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.798
 Average Recall     (AR) @[ IoU=0.50:0.95 | area

In [ ]:
val_data['images']

In [ ]:
pred_data['images']

In [19]:
df1 = pd.DataFrame(metrics_by_domain).T
df_sorted = df1.sort_index(axis=1)
df_sorted.to_excel('view/yolo/eval_domain_model.xlsx')
df2 = pd.DataFrame(metrics_by_domain).T
df_sorted = df2.sort_index(axis=1)
df_sorted.to_excel('view/yolo/eval_domain_rule.xlsx')
df3 = pd.concat([df1,df2])
df3.sort_index(axis=1).to_excel('view/yolo/eval_domain.xlsx')

## TPBank

In [ ]:
gt_path = '/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/error_analysis/tpbank/filtered_groundtruth_coco.json'
rule_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/error_analysis/tpbank/filtered_rule_predictions.json'
model_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/error_analysis/tpbank/filtered_model_predictions.json'

In [2]:
# gt_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/coco_v1/test.json'
# rule_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/fine_tuning_vgt/error_analysis/tpbank/filtered_rule_predictions.json'
# model_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/doclaynet_9k/model_infer/yolo/test.json'

In [3]:


with open(gt_path) as f:
    val_data = json.load(f)

# Load the COCO format predictions
with open(model_pred_path) as f:
    model_preds = json.load(f)

with open(rule_pred_path) as f:
    rule_preds = json.load(f)

# pred_data = {
#     'images': test['images'],
#     'annotations': model_preds,
#     'categories': test['categories']
# }

scanned_pages_id=[]
domain = 'model-tpbank'   
conf_mat, precision, recall, labels = eval(val_data,model_preds,ignore_image_ids=scanned_pages_id)
mAP = get_mAP(val_data,model_preds,ignore_image_ids=scanned_pages_id)
metrics_by_domain_model = {}
metrics_by_domain_model[domain] = {}

for label,prec,rec in zip(labels,precision,recall):
    metrics_by_domain_model[domain][f'precision-{label}']=prec
    metrics_by_domain_model[domain][f'recall-{label}']=rec
    
metrics_by_domain_model[domain] = {**metrics_by_domain_model[domain],
                                **mAP['bbox']}
with open('view/tpbank/eval_domain_model.json', 'w') as f:
    json.dump(metrics_by_domain_model,f, ensure_ascii=False, indent=4)
    
domain = 'rule-tpbank'   
conf_mat, precision, recall, labels = eval(val_data,rule_preds,ignore_image_ids=scanned_pages_id)
mAP = get_mAP(val_data,rule_preds,ignore_image_ids=scanned_pages_id)

# rule
metrics_by_domain_rule = {}
metrics_by_domain_rule[domain] = {}

for label,prec,rec in zip(labels,precision,recall):
    metrics_by_domain_rule[domain][f'precision-{label}']=prec
    metrics_by_domain_rule[domain][f'recall-{label}']=rec
    
metrics_by_domain_rule[domain] = {**metrics_by_domain_rule[domain],
                                **mAP['bbox']}
with open('view/tpbank/eval_domain_rule.json', 'w') as f:
    json.dump(metrics_by_domain_rule,f, ensure_ascii=False, indent=4)


loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
926
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=1.32s).
Accumulating evaluation results...
DONE (t=0.14s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.825
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.919
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.874
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.423
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.629
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.834
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.499
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.816
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | max

In [4]:
df1 = pd.DataFrame(metrics_by_domain_model).T
df_sorted = df1.sort_index(axis=1)
df_sorted.to_excel('eval_domain_model.xlsx')
df2 = pd.DataFrame(metrics_by_domain_rule).T
df_sorted = df2.sort_index(axis=1)
df_sorted.to_excel('eval_domain_rule.xlsx')
df3 = pd.concat([df1,df2])
df3.sort_index(axis=1).to_excel('eval_domain.xlsx')

In [ ]:
domain = 'rule-tpbank'   
ious = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
conf_mat, precision, recall, labels = eval(val_data,rule_preds,ignore_image_ids=scanned_pages_id)
mAP = get_mAP(val_data,rule_preds,ignore_image_ids=scanned_pages_id)

# rule
metrics_by_domain_rule = {}
metrics_by_domain_rule[domain] = {}

for label,prec,rec in zip(labels,precision,recall):
    metrics_by_domain_rule[domain][f'precision-{label}']=prec
    metrics_by_domain_rule[domain][f'recall-{label}']=rec
    
metrics_by_domain_rule[domain] = {**metrics_by_domain_rule[domain],
                                **mAP['bbox']}
with open('view/tpbank/eval_domain_rule.json', 'w') as f:
    json.dump(metrics_by_domain_rule,f, ensure_ascii=False, indent=4)


# QMS

In [ ]:
gt_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/layout/test.json'
rule_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/layout/rule_based_coco.json'
model_pred_path = '/home/vinhvq11/Desktop/PDF_extraction/data_process/benchmarks/qms_benchmark/layout/coco_instances_results.json'

In [ ]:


with open(gt_path) as f:
    val_data = json.load(f)

# Load the COCO format predictions
with open(model_pred_path) as f:
    model_preds = json.load(f)

with open(rule_pred_path) as f:
    rule_preds = json.load(f)

model_preds = {
    'images': val_data['images'],
    'annotations': model_preds,
    'categories': val_data['categories']
}

scanned_pages_id=[]
domain = 'model-qms'   
conf_mat, precision, recall, labels = eval(val_data,model_preds,ignore_image_ids=scanned_pages_id)
mAP = get_mAP(val_data,model_preds,ignore_image_ids=scanned_pages_id)
metrics_by_domain_model = {}
metrics_by_domain_model[domain] = {}

for label,prec,rec in zip(labels,precision,recall):
    metrics_by_domain_model[domain][f'precision-{label}']=prec
    metrics_by_domain_model[domain][f'recall-{label}']=rec
    
metrics_by_domain_model[domain] = {**metrics_by_domain_model[domain],
                                **mAP['bbox']}
with open('view/qms/eval_domain_model.json', 'w') as f:
    json.dump(metrics_by_domain_model,f, ensure_ascii=False, indent=4)
    
domain = 'rule-qms'   
conf_mat, precision, recall, labels = eval(val_data,rule_preds,ignore_image_ids=scanned_pages_id)
mAP = get_mAP(val_data,rule_preds,ignore_image_ids=scanned_pages_id)

# rule
metrics_by_domain_rule = {}
metrics_by_domain_rule[domain] = {}

for label,prec,rec in zip(labels,precision,recall):
    metrics_by_domain_rule[domain][f'precision-{label}']=prec
    metrics_by_domain_rule[domain][f'recall-{label}']=rec
    
metrics_by_domain_rule[domain] = {**metrics_by_domain_rule[domain],
                                **mAP['bbox']}
with open('view/qms/eval_domain_rule.json', 'w') as f:
    json.dump(metrics_by_domain_rule,f, ensure_ascii=False, indent=4)


In [ ]:
df1 = pd.DataFrame(metrics_by_domain_model).T
df_sorted = df1.sort_index(axis=1)
df_sorted.to_excel('view/qms/eval_domain_model.xlsx')
df2 = pd.DataFrame(metrics_by_domain_rule).T
df_sorted = df2.sort_index(axis=1)
df_sorted.to_excel('view/qms/eval_domain_rule.xlsx')
df3 = pd.concat([df1,df2])
df3.sort_index(axis=1).to_excel('view/qms/eval_domain.xlsx')

In [ ]:
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

coco_gt = COCO(filtered_gt_path)

with open(fitlered_pred_path, 'r') as f:
    predictions_data = json.load(f)

general_class_name = "text_block"
specific_categories = ["text", "title", "list-item"]
category_mapping = {cat['id']: general_class_name for cat in coco_gt.loadCats(coco_gt.getCatIds()) if cat['name'] in specific_categories}

# Create a new category for the general class in the ground truth
new_categories = [cat for cat in coco_gt.dataset['categories'] if cat['name'] not in specific_categories]
general_class_id = max(cat['id'] for cat in new_categories) + 1
new_categories.append({"id": general_class_id, "name": general_class_name})

# Modify the category IDs in ground truth annotations
new_annotations = []
for ann in coco_gt.dataset['annotations']:
    if ann['category_id'] in category_mapping:
        ann['category_id'] = general_class_id
    new_annotations.append(ann)

# Create a new ground truth dataset with modified annotations
new_ground_truth = coco_gt.dataset.copy()
new_ground_truth['categories'] = new_categories
new_ground_truth['annotations'] = new_annotations


# Modify the category IDs in predictions
predictions_data['categories'] = new_categories
for ann in predictions_data['annotations']:
    if ann['category_id'] in category_mapping:
        ann['category_id'] = general_class_id